In [35]:
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin

from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry import mapping

import matplotlib.pyplot as plt

In [13]:
#load data layers
pts = gpd.read_file('/Users/Zack/0_GIS_Greece/sar_threshold_tests/3-detections-notio-20_12-22.geojson')
#pts = gpd.read_file('/Users/Zack/Downloads/3-detections-greece_25_12-20.geojson')

search_area = gpd.read_file('/Users/Zack/0_GIS_Greece/AOI/mask_search_area_50m_5km.shp')
exclusion_zones = gpd.read_file('/Users/Zack/0_GIS_Greece/exclusions/exclusions.geojson')

#re-project layers
pts = pts.to_crs({'init': 'epsg:2100'})
search_area = search_area.to_crs({'init': 'epsg:2100'})
exclusion_zones = exclusion_zones.to_crs({'init': 'epsg:2100'})

print('detections', len(pts), pts.crs)
print('search area', len(search_area), search_area.crs)
print('exclusion zones', len(exclusion_zones), exclusion_zones.crs)

detections 36 {'init': 'epsg:2100'}
search area 1 {'init': 'epsg:2100'}
exclusion zones 2 {'init': 'epsg:2100'}


In [7]:
#plt.rcParams['figure.figsize'] = (20, 10)
#ax=search_area.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#pts.plot(markersize=5, facecolor='red', ax=ax)
#exclusion_zones.plot(ax=ax)

In [12]:
%%time 
#clip points to search area
clip = sjoin(pts, search_area, how='inner', op='within')
print('detections within search area:', len(clip))

detections within search area: 17 {'init': 'epsg:2100'}
Wall time: 682 ms


In [ ]:
#plt.rcParams['figure.figsize'] = (20, 10)
#ax=search_area.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#clip.plot(markersize=5, facecolor = 'red', ax=ax)

In [16]:
%%time 
#combine all exclusion zones
exclusion_zones['Dissolve'] = 0
exclusion_zones_dis = exclusion_zones.dissolve(by='Dissolve')

2 2
Wall time: 11.9 ms


In [18]:
%%time 
#exclude points in exclusion areas
mask = ~clip.within(exclusion_zones_dis.loc[0, 'geometry'])
clip2 = clip.loc[mask]
print('detections outside of exclusion zones:', len(clip2))

detections outside of exclusion zones: 17
Wall time: 2.99 ms


In [ ]:
#ax=search_area.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#exclusion_zones_dis.plot(facecolor = 'blue', ax=ax)
#clip2.plot(markersize=5, facecolor = 'red', ax=ax)

In [19]:
%%time 
#buffer points, dissolve buffers to aggregate points near each other
buffer = gpd.GeoDataFrame(geometry = clip2.buffer(5)) #250 meters
buffer['Dissolve'] = 0
buffer_dis = buffer.dissolve(by='Dissolve')

(17, 2) (1, 1)
Wall time: 38.3 ms


In [21]:
#explode function
def explode(indf):
    outdf = gpd.GeoDataFrame(columns=indf.columns)
    for idx, row in indf.iterrows():
        if type(row.geometry) == Polygon:
            outdf = outdf.append(row,ignore_index=True)
        if type(row.geometry) == MultiPolygon:
            multdf = gpd.GeoDataFrame(columns=indf.columns)
            recs = len(row.geometry)
            multdf = multdf.append([row]*recs,ignore_index=True)
            for geom in range(recs):
                multdf.loc[geom,'geometry'] = row.geometry[geom]
            outdf = outdf.append(multdf,ignore_index=True)
    return outdf

In [25]:
%%time 
#explode polygon and generate centroids
buffer_exploded = explode(buffer_dis)      
centroids = gpd.GeoDataFrame(geometry = buffer_exploded.centroid)
print('detections after aggregation:', len(centroids))

detections after aggregation: 17
Wall time: 26.5 ms


In [30]:
%%time 
#buffer centroids and make square polygons
centroid_buffer = gpd.GeoDataFrame(geometry = centroids.buffer(500))
envelope = gpd.GeoDataFrame(geometry = centroid_buffer.envelope)

Wall time: 9.09 ms


In [ ]:
#plt.rcParams['figure.figsize'] = (20, 10)
#ax=search_area.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#centroid_buffer.plot(markersize=5, facecolor = 'black', ax=ax)
#envelope.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none', ax=ax)

In [ ]:
#plt.rcParams['figure.figsize'] = (20, 10)
#ax = centroids.loc[[0], 'geometry'].plot(markersize=10, facecolor = 'black')
#centroid_buffer.loc[[0], 'geometry'].plot(linewidth=.5, edgecolor = 'black', facecolor = 'none', ax=ax)
#envelope.loc[[0], 'geometry'].plot(linewidth=.5, edgecolor = 'black', facecolor = 'none', ax=ax)

In [31]:
#set crs
envelope.crs = {'init' :'epsg:2100'}
envelope['geometry'] = envelope['geometry'].to_crs(epsg=4326)
print(envelope.crs)
envelope.head()

{'init': 'epsg:2100'}


,geometry
0,"POLYGON ((20.09339013363861 39.64470047416836,..."
1,"POLYGON ((20.10035111442952 39.65089914025802,..."
2,"POLYGON ((20.0698315721777 39.66155676291272, ..."
3,"POLYGON ((20.06624533138624 39.66591547289801,..."
4,"POLYGON ((20.0520422626227 39.66569691431548, ..."


In [32]:
#extract lat/long for each square polygon (envelope)
coord_list = []
for i in range(len(envelope)):
    coords = mapping(envelope.geometry[i])['coordinates']
    coord_list.append(coords)

In [33]:
#function to extract and format x/y points
def coord(input_list):
    pt1, pt2, pt3, pt4, pt5 = map(list, zip(*input_list))
    x1, y1, = map(list, zip(*pt1))
    x2, y2, = map(list, zip(*pt2))
    x3, y3, = map(list, zip(*pt3))
    x4, y4, = map(list, zip(*pt4))
    x5, y5, = map(list, zip(*pt5))

    x1=str(x1).strip("[]")
    x2=str(x2).strip("[]")
    x3=str(x3).strip("[]")
    x4=str(x4).strip("[]")
    x5=str(x5).strip("[]")

    y1=str(y1).strip("[]")
    y2=str(y2).strip("[]")
    y3=str(y3).strip("[]")
    y4=str(y4).strip("[]")
    y5=str(y5).strip("[]")

    coord_group = {'x': [x1, x2, x3, x4, x5], 'y': [y1, y2, y3, y4, y5]}
    coord_group_df = pd.DataFrame(data=coord_group)
    return coord_group_df

In [38]:
#combine x/y point groups
coord_all = []
for i in range(len(coord_list)):
    coord_group = coord(coord_list[i])
    coord_group['id']=i
    coord_all.append(coord_group)
    
targets = pd.concat(coord_all)
print('total coordinates (5 per detection):', len(targets))
targets.head()

total coordinates (5 per detection): 85


,x,y,id
0,20.093390133638607,39.64470047416836,0
1,20.105017978628698,39.645091542318376,0
2,20.104512764237462,39.6540809708806,0
3,20.092883420358827,39.6536897786341,0
4,20.093390133638607,39.64470047416836,0


In [ ]:
#targets.to_csv ('/Users/Zack/Desktop/test.csv', index = None, header=True)

In [41]:
#accuracy check
#farm_sites = gpd.read_file('/Users/Zack/0_GIS_Greece/blue_bridge/fishcages_Greece_farm_polygons.shp')
farm_sites = gpd.read_file('/Users/Zack/0_GIS_Greece/sar_threshold_tests/farmsites_notio.shp')

centroids_test = centroids.copy()
centroids_test.crs = {'init' :'epsg:2100'}
centroids_test['geometry'] = centroids_test['geometry'].to_crs(epsg=4326)

matches = centroids_test.intersects(farm_sites.unary_union)
count = centroids_test.loc[matches]

detections_n = len(centroids_test)
farm_n = len(farm_sites)
matches_n = len(count)

print('total detections:', len(pts))
print('detections in search area:', detections_n)
print('farm sites:', farm_n)
print('matches:', matches_n)
print('positive matches', round(matches_n / farm_n, 3))
print('overall accuracy', round(matches_n / detections_n, 3))

#plt.rcParams['figure.figsize'] = (20, 10)
#ax=farm_sites.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#centroids_test.plot(markersize=1, facecolor = 'red', ax=ax)

total detections: 36
detections in search area: 17
farm sites: 21
matches: 14
positive matches 0.667
overall accuracy 0.824
